In [ ]:
# !pip install vit-keras
!pip install tensorflow-addons
from tensorflow.keras.preprocessing.image import img_to_array


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install vit_keras

!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=e2f5b1ff1901499c5f081a6f5ec23c2e6cb9b64cbd3a8ab8f881a7faddcd2803
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from vit_keras import vit

# Load pre-trained ViT model
model = vit.vit_b16(
    image_size=256,  # Size of the input images
    pretrained=True  # Use pre-trained weights
)

# Load image and preprocess it
image = load_img('modified.jpeg', target_size=(256, 256))
image = img_to_array(image)
image = preprocess_input(image)
image = image.reshape(1, 256, 256, 3)

# Make predictions
preds = model.predict(image)





/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


347502902/347502902 [==============================] - 3s 0us/step


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 16, 16
  warnings.warn(


FileNotFoundError: ignored

In [ ]:
import vit_keras

print(dir(vit_keras.vit))

# print(preds)

AttributeError: ignored

In [ ]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d kaustubhchaudhari/pubfig-dataset-256x256-jpg
!unzip pubfig-dataset-256x256-jpg.zip

In [ ]:
import os
import shutil
import random

# Set the path to your original dataset folder
original_dataset_path = '/content/CelebDataProcessed'

# Set the path to the folder where you want to create the train and test folders
base_dir = '/content/Facesfolder'
os.makedirs(base_dir, exist_ok=True)

# Set the percentage of images to use for the test set
test_split_percentage = 20

# Create the train and test folders
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Iterate through each actor folder in the original dataset folder
for actor_name in os.listdir(original_dataset_path):
    actor_dir = os.path.join(original_dataset_path, actor_name)
    if not os.path.isdir(actor_dir):
        continue

    # Create a folder for the actor in the train and test directories
    train_actor_dir = os.path.join(train_dir, actor_name)
    test_actor_dir = os.path.join(test_dir, actor_name)
    os.makedirs(train_actor_dir, exist_ok=True)
    os.makedirs(test_actor_dir, exist_ok=True)

    # Iterate through each image in the actor folder
    actor_images = os.listdir(actor_dir)
    random.shuffle(actor_images)  # Shuffle the images randomly
    test_split_index = int(len(actor_images) * (test_split_percentage / 100))
    train_images = actor_images[test_split_index:]
    test_images = actor_images[:test_split_index]

    # Copy the train images to the train actor folder
    for image_name in train_images:
        image_path = os.path.join(actor_dir, image_name)
        target_path = os.path.join(train_actor_dir, image_name)
        shutil.copy(image_path, target_path)

    # Copy the test images to the test actor folder
    for image_name in test_images:
        image_path = os.path.join(actor_dir, image_name)
        target_path = os.path.join(test_actor_dir, image_name)
        shutil.copy(image_path, target_path)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from vit_keras import vit

# Prepare the dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    './Facesfolder/train',
    target_size=(256, 256),
    batch_size=16,
    classes=['Abhishek Bachan','Alex Rodriguez','Colin Powell','Cristiano Ronaldo','Daisy Fuentes','Daniel Craig','Rosario Dawson'],
    class_mode='categorical')

test_set = test_datagen.flow_from_directory(
    'Facesfolder/test',
    classes=['Abhishek Bachan','Alex Rodriguez','Colin Powell','Cristiano Ronaldo','Daisy Fuentes','Daniel Craig','Rosario Dawson'],

    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical')

model = vit.vit_b16(
    image_size=256,  
    pretrained=True,  # Use pre-trained weights
)
model.trainable = False #(freezing weights)

num_classes = len(train_set.class_indices)
top_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
model = tf.keras.Sequential([model, top_layer])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, epochs=5, validation_data=test_set)


loss, accuracy = model.evaluate(test_set)
print('Test accuracy:', accuracy)


Found 386 images belonging to 7 classes.
Found 93 images belonging to 7 classes.
Epoch 1/5
25/25 [==============================] - 580s 23s/step - loss: 1.7480 - accuracy: 0.4041 - val_loss: 1.0283 - val_accuracy: 0.6559
Epoch 2/5
25/25 [==============================] - 489s 20s/step - loss: 0.9406 - accuracy: 0.6943 - val_loss: 0.6405 - val_accuracy: 0.7849
Epoch 3/5
25/25 [==============================] - 487s 20s/step - loss: 0.6498 - accuracy: 0.8057 - val_loss: 0.5751 - val_accuracy: 0.7742
Epoch 4/5
25/25 [==============================] - 492s 20s/step - loss: 0.5081 - accuracy: 0.8446 - val_loss: 0.5912 - val_accuracy: 0.7849
Epoch 5/5
12/12 [==============================] - 80s 7s/step - loss: 0.4266 - accuracy: 0.8495
Test accuracy: 0.8494623899459839


In [ ]:
!unzip "/content/CloakedImages.zip" -d ./Test

In [ ]:
import numpy as np


test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    '/content/Test/Test2k_images/Facesfolder_per20/test',
    classes=['Abhishek Bachan','Alex Rodriguez','Colin Powell','Cristiano Ronaldo','Daisy Fuentes','Daniel Craig','Rosario Dawson'],

    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical')


filenames = test_set.filenames
true_labels = test_set.classes

predicted_probabilities = model.predict(test_set)
predicted_labels = np.argmax(predicted_probabilities, axis=1)

for i in range(len(filenames)):
    print('Filename:', filenames[i])
    print('True label:', true_labels[i])
    print('Predicted label:', predicted_labels[i])




Found 142 images belonging to 7 classes.
18/18 [==============================] - 131s 7s/step
Filename: Alex Rodriguez/13.jpg
True label: 1
Predicted label: 5
Filename: Alex Rodriguez/13_cloaked.png
True label: 1
Predicted label: 3
Filename: Alex Rodriguez/16.jpg
True label: 1
Predicted label: 5
Filename: Alex Rodriguez/16_cloaked.png
True label: 1
Predicted label: 1
Filename: Alex Rodriguez/2.jpg
True label: 1
Predicted label: 6
Filename: Alex Rodriguez/26.jpg
True label: 1
Predicted label: 2
Filename: Alex Rodriguez/26_cloaked.png
True label: 1
Predicted label: 6
Filename: Alex Rodriguez/28.jpg
True label: 1
Predicted label: 2
Filename: Alex Rodriguez/28_cloaked.png
True label: 1
Predicted label: 6
Filename: Alex Rodriguez/29.jpg
True label: 1
Predicted label: 2
Filename: Alex Rodriguez/29_cloaked.png
True label: 1
Predicted label: 5
Filename: Alex Rodriguez/2_cloaked.png
True label: 1
Predicted label: 2
Filename: Alex Rodriguez/43.jpg
True label: 1
Predicted label: 5
Filename: Alex

In [ ]:
!unzip "/content/test.zip" -d ./OnlyCloaked

In [ ]:
import numpy as np


test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    '/content/OnlyCloaked/test',
    classes=['Abhishek Bachan','Alex Rodriguez','Colin Powell','Cristiano Ronaldo','Daisy Fuentes','Daniel Craig','Rosario Dawson'],

    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical')


filenames = test_set.filenames
true_labels = test_set.classes

loss, accuracy = model.evaluate(test_set)
print('Test accuracy:', accuracy)


Found 93 images belonging to 7 classes.
12/12 [==============================] - 84s 7s/step - loss: 0.3472 - accuracy: 0.8925
Test accuracy: 0.8924731016159058
